In [99]:
climate['t'] = climate.yyyy + (climate.mm-1)/12    # timestamp in years
climate['temp'] = (climate.tmin + climate.tmax)/2  # monthly avg temp
df1 = climate.loc[(climate.station == 'Cambridge') & (climate.yyyy>=1980)]
def temp_inc(t, temp):
    model = sklearn.linear_model.LinearRegression()
    model.fit(np.column_stack([np.sin(2*π*t), np.cos(2*π*t), t-2000]), temp)
    return model

model = temp_inc(df1.t, df1.temp)

alpha = model.intercept_
beta1, beta2, gamma = model.coef_

sigma = mean_squared_error(model.predict(np.column_stack([np.sin(2*π*df1.t), np.cos(2*π*df1.t), df1.t-2000])), df1.temp)

print(alpha, beta1, beta2, gamma, sigma)

10.514034451752183 -1.107716558433125 -6.560983691731712 0.035071793932324624 2.100057727817012


In [100]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
import sklearn.linear_model
π = np.pi
from sklearn.metrics import mean_squared_error

In [101]:
def tempGen(t):
    return np.random.normal(loc=model.predict(np.column_stack([np.sin(2*π*t), np.cos(2*π*t), t-2000])), scale=sigma)

In [98]:
tnew = np.linspace(1985, 2022, 1000)
gentemps = [[tempGen(t) for t in tnew] for _ in range(1000)]
def temp_inc_only(time, synthtemp):
    m = sklearn.linear_model.LinearRegression()
    m.fit(np.column_stack([np.sin(2*π*time), np.cos(2*π*time), time-2000]), synthtemp)
    return m.coef_[0][2]

tempincs = [temp_inc_only(tnew, gentemp) for gentemp in gentemps]

lo , hi = np.quantile(tempincs, [0.025 , 0.975])
print(lo, hi)

0.023248998597875527 0.04698327009036507


In [1]:
import numpy as np
import pandas
import requests, requests_cache
import urllib
import io, re
import lxml.html
import numbers
requests_cache.install_cache(backend='memory')

In [2]:
# Fetch a list of all measurement stations
stations_url = 'https://www.metoffice.gov.uk/research/climate/maps-and-data/historic-station-data'
stations = requests.get(stations_url).content
stations = lxml.html.fromstring(stations)
stations = stations.xpath("//h2[contains(text(),'List of all the historic stations')]/following-sibling::table/tbody/tr")
stations = [tr.xpath('./td/a/@href')[0] for tr in stations]

In [3]:
# Fetch historical data from each station
def parse_station_data(txt, i=None):
    ls = txt.splitlines()
    g = (i for i,l in enumerate(ls) if ' '.join(l.split()).startswith('yyyy mm tmax tmin'))
    header_line = next(g)
    d = '\n'.join([ls[header_line]+' status X1 X2 X3 X4 X5 X6'] + ls[(header_line+2):])
    df = pandas.read_csv(io.StringIO(d), delim_whitespace=True)
    df['station'] = [ls[0]] * len(df)
    s = ls[1] + ' ' + ls[2]
    m = re.search(r'Lat ([-\d.]+) Lon ([-\d.]+),? ([-\d.]+)\s?m.* amsl', s)
    assert m is not None, f"Can't read details for {i}: {s}"
    df['lat'] = [m.group(1)] * len(df)
    df['lng'] = [m.group(2)] * len(df)
    df['height'] = [m.group(3)] * len(df)
    return df
df = [requests.get(station_url).content.decode('utf8') for station_url in stations]
df = [parse_station_data(txt,i) for i,txt in enumerate(df)]

# Concatenate all the station together
df = pandas.concat(df, ignore_index=True)
# Sometimes there are comment rows, and df.yyy is string
# Sometimes there are no comment rows, and df.yyy is int
# This code removes any comment rows, safely in either case
df = df.loc[df.yyyy.astype(str) != 'Site']

# Delete some dummy columns (which were only there for rows with comments)
for i in range(1,7): del df[f'X{i}']

# Normalize some long-winded station names
for old,new in [('Heathrow (London Airport)','Heathrow'), 
                ('Lowestoft / Lowestoft Monckton Ave from Sept 2007','Lowestoft'),
                ('Whitby Coastguard / Whitby(from 2000 and $sunshine)','Whitby'),
                ('Nairn   there is a site change in 1998','Nairn'),
                ('Cambridge NIAB','Cambridge')]:
    df['station'] = np.where(df['station']==old, new, df['station'])
    
# Tidy the numerical columns, to only have numbers or NaN
def clean_num(x):
    if isinstance(x, numbers.Number):
        return x
    elif x == '---':
        return np.nan
    else:
        return float(re.match(r'([-\d.]+)', x).group(1))
df['tmin'] = np.array([clean_num(x) for x in df['tmin']])
df['tmax'] = np.array([clean_num(x) for x in df['tmax']])
df['yyyy'] = np.array([clean_num(x) for x in df['yyyy']]).astype(np.int)
df['mm'] = np.array([clean_num(x) for x in df['mm']]).astype(np.int)

climate = df

/var/folders/ll/mm0mcnc52699y4zxxcgq32sw0000gn/T/ipykernel_77216/1127624974.py:47: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df['yyyy'] = np.array([clean_num(x) for x in df['yyyy']]).astype(np.int)
/var/folders/ll/mm0mcnc52699y4zxxcgq32sw0000gn/T/ipykernel_77216/1127624974.py:48: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the prec